### Import Dependencies

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import json
import requests
from Meetup_key import meetup_key

### Define functions to fetch data from Meetup API

In [ ]:
def getCities():
    response = requests.get(f"{MainUrl}&sign=true&photo-host=public&country={country}&state={state}&page={num_cities}")
    data = response.json()
    for i in range(0, len(data['results'])):
        cities_out.append(data['results'][i])

In [ ]:
def getGroups():
    response = requests.get(f"{MainUrl}&sign=true&photo-host=public&country={country}&zip={zip_code}&page={num_groups}")
    data = response.json()    
    for i in range(0, len(data['results'])):
        try:
            category_out.append(data['results'][i]['category'])
            group_out.append(data['results'][i])
        except KeyError:
            print("Bad group")
            bad_group.append(data['results'][i]['id'])        
    print([len(group_out), len(category_out)])

In [ ]:
def getMembers():
    response = requests.get(f"{MainUrl}&sign=true&photo-host=public&group_id={group_id}&page={num_members}&offset={page}")
    data = response.json()    
    for i in range(0, len(data['results'])):
        try:
            member_out.append(data['results'][i])
            id_out.append(group_id)
        except KeyError:
            print("Bad member")
            #bad_member.append(data['results'][i]['id'])        
    print([len(member_out), len(id_out)])

In [ ]:
def loopPages():
    page = 0
    query = f"{MainUrl}&sign=true&photo-host=public&group_id={group_id}&page={num_members}&offset={page}"
    response = requests.get(query)
    data = response.json() 
    print([data['meta']['total_count'], group_id])
    for page in range(0, (round(data['meta']['total_count']/200)+1)):
        query = f"{MainUrl}&sign=true&photo-host=public&group_id={group_id}&page={num_members}&offset={page}"
        response = requests.get(query)
        data = response.json() 
        #print(query)
        for i in range(0, len(data['results'])):
            try:
                member_out.append(data['results'][i])
                id_out.append(group_id)
            except KeyError:
                print("error")
        

In [2]:
def loopPages():
    page = 0
    query = f"{MainUrl}&sign=true&photo-host=public&group_id={group_id}&page={num_members}&offset={page}"
    response = requests.get(query)
    data = response.json() 
    print([data['meta']['total_count'], group_id])
    for page in range(0, (round(data['meta']['total_count']/200)+1)):
        query = f"{MainUrl}&sign=true&photo-host=public&group_id={group_id}&page={num_members}&offset={page}"
        response = requests.get(query)
        data = response.json() 
        #print(query)
        for i in range(0, len(data['results'])):
            try:
                data['results'][i].update({'group_id' :f"{group_id}"})
                member_out.append(data['results'][i])
                #id_out.append(group_id)
            except KeyError:
                print("error")
        

### Define static parameters and output lists for getCities function

In [ ]:
MainUrl = f"https://api.meetup.com/2/cities?key={meetup_key}"
country = "us"; num_cities = 10; cities_out = []

states1 = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL',
          'IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO']

states2 = ['MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA',
          'RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

### Execute getCities function on both state lists

In [ ]:
for state in states1
    getCities()

In [ ]:
for state in states2:
    getCities()

### Convert result list into Dataframe and export to csv file

In [ ]:
cities_df = pd.DataFrame.from_dict(cities_out)
cities_df.to_csv('top_ten_cities.csv')

### Create list of Zip codes to use in secondary API call

In [ ]:
zip_code = cities_df['zip']; zip_code1 = zip_code[0:100]; zip_code2 = zip_code[100:200]
zip_code3 = zip_code[200:300]; zip_code4 = zip_code[300:400]; zip_code5 = zip_code[400:500]

### Define static parameters and output lists for getGroups function

In [ ]:
country = "us"
num_groups = 50
group_out = []
bad_group = []
category_out = []
MainUrl = f"https://api.meetup.com/2/groups?key={meetup_key}"

### Execute getGroups function for each zip code list

In [ ]:
for zip_code in zip_code1:
    getGroups()

In [ ]:
for zip_code in zip_code2:
    getGroups()

In [ ]:
for zip_code in zip_code3:
    getGroups()

In [ ]:
for zip_code in zip_code4:
    getGroups()

In [ ]:
for zip_code in zip_code5:
    getGroups()

### Convert to data frame, combine with categories, remove unnecessary columns and export to csv

In [ ]:
group_out_df = pd.DataFrame.from_dict(group_out)
groups_clean = group_out_df.drop(['category', 'group_photo', 'organizer',
                                       'link', 'utc_offset', 'topics'], axis=1)
categories = pd.DataFrame.from_dict(category_out)
categories = categories.rename(index='str', columns={'id': 'cat_id', 'name': 'cat_name', 'shortname': 'cat_shortname'})
combo_df = pd.concat([groups_clean, categories], axis=1)
combo_df = combo_df.dropna()
combo_df.to_csv('groups+categories.csv')

### Read in 'groups+categories.csv' then filter the results to show only public groups

In [3]:
# The getGroups API calls resulted in duplicate group records, nearly doubling the size of the dataset. To fix this, drop the 
# duplicate rows and reset the Dataframe index 

combo_df = pd.read_csv('data_output\groups+categories.csv', dtype={'id': 'Int64','members': 'Int64', 'cat_id': 'Int64', 'lat': 'float64', 
                                                       'lon': 'float64', 'rating': 'float64' })
public_groups = combo_df[combo_df['visibility']=='public'].drop_duplicates()
public_groups = public_groups.reset_index(drop=True)
#public_groups

### Determine the max amount of members then calculate the max page range for 'page' API parameter

In [4]:
max_members = public_groups['members'].max()
max_pages = round(max_members/200)+1
max_pages

144

### Isolate group ids, convert to the integer data type, and divide the group ids into sets of 1000

In [11]:
group_id = public_groups['id']; group_id = group_id.astype('Int64');
group_id1 = group_id[0:500] 
group_id2 = group_id[500:1000]
group_id3 = group_id[1000:1500]
group_id4 = group_id[1500:2000] 
group_id5 = group_id[2000:2500]
group_id6 = group_id[2500:3000]
group_id7 = group_id[3000:3500]
group_id8 = group_id[3500:4000]
group_id9 = group_id[4000:4500]
group_id10 = group_id[4500:5000]

### Define static parameters and output lists for getMembers function

In [6]:
country = "us"
num_members = 200
page = 0
member_out = []
#member_temp = []
#id_temp = []
topics_out = []
bad_member = []
#category_out = []
MainUrl = f"https://api.meetup.com/2/members?key={meetup_key}"

In [7]:
country = "us"
num_members = 200
page = 0
member_out = []
#member_temp = []
#id_temp = []
topics_out = []
bad_member = []
#category_out = []
MainUrl = f"https://api.meetup.com/2/members?key={meetup_key}"
###################################################################################
for group_id in group_id1:
    loopPages()
###################################################################################
member_out_df = pd.DataFrame.from_dict(member_out)
member_out_df['joined'] = pd.to_datetime(member_out_df['joined'], unit='ms')
member_out_df['visited'] = pd.to_datetime(member_out_df['visited'], unit='ms')
member_out_df.to_csv('data_output\member_data1.csv') 

[1259, 233403]
[595, 901780]
[2234, 1378740]
[1566, 1661726]
[1693, 1785501]
[1091, 1805016]
[386, 1907081]
[662, 1959361]
[373, 2139191]
[1131, 3621552]
[1471, 3772782]
[250, 4525562]
[909, 6612252]
[206, 8279522]
[221, 8950622]
[1898, 9286572]
[1189, 9328932]
[259, 11726642]
[279, 12028242]
[183, 12419742]
[1293, 12481022]
[436, 14590822]
[708, 15069832]
[933, 15871952]
[3001, 16638092]
[293, 17029332]
[242, 17334672]
[546, 17606582]
[764, 17655062]
[668, 18099562]
[1061, 18191567]
[600, 18323657]
[454, 18371362]
[50, 18388762]
[357, 18497452]
[2383, 18508923]
[2448, 18531434]
[1298, 18560346]
[1605, 27214]
[1701, 12843]
[41, 296563]
[32, 311990]
[967, 323739]
[3139, 530814]
[1095, 541815]
[388, 1006435]
[1683, 1390468]
[1797, 1417163]
[883, 1491258]
[480, 1600438]
[320, 1727534]
[865, 1736241]
[496, 1797675]
[649, 1995301]
[140, 2087031]
[44, 3062062]
[247, 3326612]
[801, 3507942]
[333, 7037522]
[161, 7883842]
[180, 9719722]
[967, 13107862]
[79, 15445882]
[206, 18005132]
[406, 18397

NameError: name 'member_out_df' is not defined

In [10]:
country = "us"
num_members = 200
page = 0
member_out = []
topics_out = []
bad_member = []
MainUrl = f"https://api.meetup.com/2/members?key={meetup_key}"
###################################################################################
for group_id in group_id2:
    loopPages()
###################################################################################    
member_out_df = pd.DataFrame.from_dict(member_out)
member_out_df['joined'] = pd.to_datetime(member_out_df['joined'], unit='ms')
member_out_df['visited'] = pd.to_datetime(member_out_df['visited'], unit='ms')
member_out_df.to_csv('data_output\member_data2.csv')    

[220, 19292332]
[299, 21067906]
[324, 22649509]
[131, 22852502]
[210, 24118287]
[20, 27241728]
[39, 28293336]
[52, 28296456]
[38, 28296614]
[37, 29322885]
[50, 19566911]
[14, 25558472]
[0, 29237695]
[48, 29757573]
[25, 30486900]
[1246, 18550630]
[111, 21252012]
[77, 21634245]
[3, 21959747]
[96, 57056]
[3670, 120211]
[1363, 163801]
[3714, 63890]
[7023, 110238]
[2368, 86809]
[1134, 32542]
[520, 67804]
[4199, 55321]
[5564, 54658]
[1774, 126457]
[3408, 105552]
[1610, 33171]
[517, 171767]
[883, 137696]
[474, 36467]
[1327, 157084]
[3288, 143588]
[851, 170013]
[1398, 52591]
[352, 57700]
[1130, 134732]
[1040, 64090]
[1325, 120295]
[45, 130720]
[344, 104001]
[3021, 109708]
[2200, 159753]
[2006, 179376]
[1174, 180480]
[1388, 194380]
[3363, 205539]
[3540, 208211]
[354, 214789]
[608, 216604]
[1021, 229120]
[3370, 230552]
[512, 233317]
[497, 2657]
[362, 54614]
[104, 59029]
[76, 97579]
[259, 27422]
[6272, 61234]
[1475, 63930]
[202, 27514]
[1078, 146031]
[28, 63927]
[261, 131406]
[757, 83230]
[145, 8

In [12]:
country = "us"
num_members = 200
page = 0
member_out = []
topics_out = []
bad_member = []
MainUrl = f"https://api.meetup.com/2/members?key={meetup_key}"
###################################################################################
for group_id in group_id3:
    loopPages()
###################################################################################    
member_out_df = pd.DataFrame.from_dict(member_out)
member_out_df['joined'] = pd.to_datetime(member_out_df['joined'], unit='ms')
member_out_df['visited'] = pd.to_datetime(member_out_df['visited'], unit='ms')
member_out_df.to_csv('data_output\member_data3.csv')    

[2081, 267925]
[1832, 272057]
[2924, 288027]
[1650, 308139]
[68, 316744]
[523, 337555]
[1408, 342580]
[1809, 343899]
[480, 358912]
[725, 365465]
[16, 368210]
[2202, 370628]
[540, 384872]
[593, 392518]
[155, 450914]
[1342, 482725]
[520, 490701]
[1505, 494922]
[1682, 501417]
[1176, 506423]
[729, 506822]
[49, 219949]
[288, 543243]
[553, 556137]
[2902, 582937]
[1198, 597758]
[2872, 639940]
[6445, 654743]
[714, 677346]
[1266, 804306]
[470, 836316]
[227, 920321]
[341, 926502]
[603, 1540019]
[875, 1551111]
[1542, 1569894]
[134, 1586532]
[450, 1573159]
[103, 1630043]
[430, 1692096]
[2209, 1767437]
[34, 1773593]
[37, 1808325]
[106, 1809737]
[426, 3337872]
[166, 3761482]
[1048, 3822522]
[940, 3866032]
[6, 3981642]
[29, 4327642]
[138, 5955332]
[501, 6087402]
[50, 12736392]
[318, 13047152]
[1261, 13886192]
[166, 15464112]
[281, 16253542]
[160, 17015342]
[336, 17093112]
[122, 17155422]
[118, 17167892]
[440, 17286882]
[362, 18025742]
[86, 18178959]
[66, 18236389]
[500, 18257874]
[225, 18307576]
[425

KeyboardInterrupt: 

In [ ]:
country = "us"
num_members = 200
page = 0
member_out = []
topics_out = []
bad_member = []
MainUrl = f"https://api.meetup.com/2/members?key={meetup_key}"
###################################################################################
for group_id in group_id4:
    loopPages()
###################################################################################    
member_out_df = pd.DataFrame.from_dict(member_out)
member_out_df['joined'] = pd.to_datetime(member_out_df['joined'], unit='ms')
member_out_df['visited'] = pd.to_datetime(member_out_df['visited'], unit='ms')    
member_out_df.to_csv('data_output\member_data4.csv')    

In [ ]:
country = "us"
num_members = 200
page = 0
member_out = []
topics_out = []
bad_member = []
MainUrl = f"https://api.meetup.com/2/members?key={meetup_key}"
###################################################################################
for group_id in group_id5:
    loopPages()
###################################################################################
member_out_df = pd.DataFrame.from_dict(member_out)
member_out_df['joined'] = pd.to_datetime(member_out_df['joined'], unit='ms')
member_out_df['visited'] = pd.to_datetime(member_out_df['visited'], unit='ms')
member_out_df.to_csv('data_output\member_data5.csv')    

In [ ]:
country = "us"
num_members = 200
page = 0
member_out = []
topics_out = []
bad_member = []
MainUrl = f"https://api.meetup.com/2/members?key={meetup_key}"
###################################################################################
for group_id in group_id6:
    loopPages()
###################################################################################
member_out_df = pd.DataFrame.from_dict(member_out)
member_out_df['joined'] = pd.to_datetime(member_out_df['joined'], unit='ms')
member_out_df['visited'] = pd.to_datetime(member_out_df['visited'], unit='ms')
member_out_df.to_csv('data_output\member_data6.csv')    

In [ ]:
country = "us"
num_members = 200
page = 0
member_out = []
topics_out = []
bad_member = []
MainUrl = f"https://api.meetup.com/2/members?key={meetup_key}"
###################################################################################
for group_id in group_id7:
    loopPages()
###################################################################################
member_out_df = pd.DataFrame.from_dict(member_out)
member_out_df['joined'] = pd.to_datetime(member_out_df['joined'], unit='ms')
member_out_df['visited'] = pd.to_datetime(member_out_df['visited'], unit='ms')
member_out_df.to_csv('data_output\member_data7.csv')    

In [ ]:
country = "us"
num_members = 200
page = 0
member_out = []
topics_out = []
bad_member = []
MainUrl = f"https://api.meetup.com/2/members?key={meetup_key}"
###################################################################################
for group_id in group_id8:
    loopPages()
###################################################################################
member_out_df = pd.DataFrame.from_dict(member_out)
member_out_df['joined'] = pd.to_datetime(member_out_df['joined'], unit='ms')
member_out_df['visited'] = pd.to_datetime(member_out_df['visited'], unit='ms')    
member_out_df.to_csv('data_output\member_data8.csv')    

In [ ]:
country = "us"
num_members = 200
page = 0
member_out = []
topics_out = []
bad_member = []
MainUrl = f"https://api.meetup.com/2/members?key={meetup_key}"
###################################################################################
for group_id in group_id9:
    loopPages()
###################################################################################
member_out_df = pd.DataFrame.from_dict(member_out)
member_out_df['joined'] = pd.to_datetime(member_out_df['joined'], unit='ms')
member_out_df['visited'] = pd.to_datetime(member_out_df['visited'], unit='ms')
member_out_df.to_csv('data_output\member_data9.csv')    

In [ ]:
country = "us"
num_members = 200
page = 0
member_out = []
topics_out = []
bad_member = []
MainUrl = f"https://api.meetup.com/2/members?key={meetup_key}"
###################################################################################
for group_id in group_id10:
    loopPages()
###################################################################################
member_out_df = pd.DataFrame.from_dict(member_out)
member_out_df['joined'] = pd.to_datetime(member_out_df['joined'], unit='ms')
member_out_df['visited'] = pd.to_datetime(member_out_df['visited'], unit='ms')    
member_out_df.to_csv('data_output\member_data10.csv')    

In [ ]:
member_out_df = pd.DataFrame.from_dict(member_out)
member_out_df['joined'] = pd.to_datetime(member_out_df['joined'], unit='ms')
member_out_df['visited'] = pd.to_datetime(member_out_df['visited'], unit='ms')
member_out_df